## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

This is my project path, you need to change your path before run it.\
And the KMP is my computer setting, you could not to change it.

In [ ]:
import os
HOME = "D:\Document\Study\SFU\CMPT732\YOLOv8-SmallObj-Garbage"
print(HOME)
os.chdir(HOME)
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'


## Check the YOLO status

pip show is check the yolo is install successful

In [ ]:
!pip show ultralytics

If yolo is not install,please run blow section.

In [ ]:
# Git clone method (for development)

%cd {HOME}/ultralytics
!pip install -e .

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image
from IPython import display
import ultralytics

display.clear_output()
ultralytics.checks()

Ultralytics YOLOv8.0.196  Python-3.8.16 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
Setup complete  (32 CPUs, 95.7 GB RAM, 1272.7/1713.0 GB disk)


Create the yaml file

In [ ]:
import yaml

data_yaml = dict(
    train ='TACO-2\\train',
    val ='TACO-2\\valid',
    test='TACO-2\\test',
    nc =18,
    names =['Aluminium foil', 'Bottle cap', 'Bottle', 'Broken glass', 'Can',
            'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter', 'Other plastic',
            'Paper', 'Plastic bag - wrapper', 'Plastic container', 'Pop tab',
            'Straw', 'Styrofoam piece', 'Unlabeled litter']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

Build a new model from scratch

In [ ]:
model = YOLO(f'yolov8n.yaml')
model_name = "Normal"

In [ ]:
model = YOLO(f'yolov8n-CBAM.yaml')
model_name = "CBAM"

In [ ]:
model = YOLO(f'yolov8n-SPD.yaml')
model_name = "SPD"

In [ ]:
model = YOLO(f'yolov8n-SPD-CBAM.yaml')
model_name = "SPD-CBAM"

In [ ]:
model = YOLO(f'yolov8n-P2.yaml')
model_name = "Mult-Head"

In [ ]:
model = YOLO(f'yolov8n-SPD-P2.yaml')
model_name = "Mult-Head-SPD"

In [ ]:
model = YOLO(f'yolov8n-CBAM-P2.yaml')
model_name = "CBAM-P2"

In [ ]:
model = YOLO(f'yolov8n-SPD-CBAM-P2.yaml')
model_name = "SPD-CBAM-P2"

Load the model.

In [ ]:
model_weight_path = f'{HOME}\small_grabage\{model_name}\weights\last.pt'

model = YOLO(model_weight_path)
print(model)

## Custom Training

In [ ]:
!python -m tensorboard.main --logdir='small_grabage/'

In [ ]:
project_name = "small_grabage"
dataset_location = "D:\Document\Study\SFU\CMPT732\YOLOv8-SmallObj-Garbage\datasets\TACO-2"
# !yolo task=detect mode=train model=yolov8x.pt data=data.yaml epochs=5 imgsz=400 plots=True

In [ ]:
resume = True
results = model.train(data='data.yaml', epochs=1000, imgsz=256,batch=16,workers=8,project=project_name,name=model_name,resume=resume)

## Validate Custom Model

In [ ]:
%cd {HOME}
model.val()

In [ ]:
model.export()

In [ ]:
model.info()

YOLOv8n-SPD-CBAM-P2 summary (fused): 235 layers, 3719738 parameters, 0 gradients, 69.0 GFLOPs


(235, 3719738, 0, 68.9879616)

## Inference with Custom Model

In [ ]:
source_image_path = f"{dataset_location}//test//images"

In [ ]:
results = model.predict(source=source_image_path,save=True,save_dir=f'{HOME}\results\\{model_name}') # Display preds. Accepts all YOLO predict arguments

**NOTE:** Let's take a look at few results.

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[3:5]:
      display(Image(filename=image_path, width=600))
      print("\n")

## SAHI Framework
### Import required modules:


In [ ]:
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict

Set model and directory parameters:

In [ ]:
model_type = "yolov8"
model_path = model_weight_path
model_device = "cudo:0" # or 'cuda:0'
model_confidence_threshold = 0.2

slice_height = 256
slice_width = 256
overlap_height_ratio = 0.7
overlap_width_ratio = 0.7

Perform sliced inference on given folder:


In [ ]:
model_name_list = [
    #"Baseline","Baseline-P2",  "SPD","SPD-P2", "CBAM","CBAM-P2","SPD-CBAM",    "SPD-CBAM-P2"]
    "SPD-CBAM-P2",
    "SPD-CBAM",
    "SPD",
    "SPD-P2",
    "CBAM",
    "CBAM-P2",
    "Baseline",
    "Baseline-P2"
]

for model_name in model_name_list:
  model_weight_path = f'{HOME}\\small_grabage\\{model_name}\\weights\\best.pt'
  # print(model_weight_path)
  model = YOLO(model_weight_path)

  model.info()
  # model.predict(source=source_image_path,save=True,save_dir=f'{HOME}\results\\{model_name}') # Display preds. Accepts all YOLO predict arguments

  # predict(
  #     model_type=model_type,
  #     model_path=model_weight_path,
  #     model_device=model_device,
  #     model_confidence_threshold=model_confidence_threshold,
  #     source=source_image_path,
  #     slice_height=slice_height,
  #     slice_width=slice_width,
  #     overlap_height_ratio=overlap_height_ratio,
  #     overlap_width_ratio=overlap_width_ratio,
  # )

In [ ]:
runs_path = f"{HOME}\\runs\\detect"
model_name_list = ["Baseline","Baseline-P2",  "SPD","SPD-P2", "CBAM","CBAM-P2","SPD-CBAM",    "SPD-CBAM-P2"]
dir_list = []
for dir in os.listdir(runs_path):
        dir = os.path.join(runs_path, dir)
        if(dir.split('predict')[-1]==""):
          path111 = os.path.join(dir.split('predict')[0],"predict1")
          os.rename(dir,path111)
        if os.path.isdir(dir):
          dir_list.append(dir)
dir_list.sort(key=lambda x: int(x.split('predict')[-1]))

if len(model_name_list) != len(dir_list):
    print("Error: The number of model names does not match the number of directories.")
else:
    for i, dir in enumerate(dir_list):
        new_name = os.path.join(runs_path, model_name_list[i])
        os.rename(dir, new_name)

        print(f"Renamed {dir} to {new_name}")

In [ ]:
runs_path = f"{HOME}\\runs\\predict"
model_name_list = ["Baseline","Baseline-P2",  "SPD","SPD-P2", "CBAM","CBAM-P2","SPD-CBAM",    "SPD-CBAM-P2"]
dir_list = []
for dir in os.listdir(runs_path):
        dir = os.path.join(runs_path, dir)
        if(dir.split('exp')[-1]==""):
          path111 = os.path.join(dir.split('exp')[0],"exp1")
          os.rename(dir,path111)
        if os.path.isdir(dir):
          dir_list.append(dir)
dir_list.sort(key=lambda x: int(x.split('exp')[-1]))

if len(model_name_list) != len(dir_list):
    print("Error: The number of model names does not match the number of directories.")
else:
    for i, dir in enumerate(dir_list):
        new_name = os.path.join(runs_path, "SAHI_"+model_name_list[i])
        os.rename(dir, new_name)

        print(f"Renamed {dir} to {new_name}")

In [ ]:
from PIL import Image, ImageDraw, ImageFont



model_name_list = [
    "Baseline",
    "Baseline-P2",
    "SAHI_Baseline",
    "SAHI_Baseline-P2",
    "SPD",
    "SPD-P2",
    "SAHI_SPD",
    "SAHI_SPD-P2",
    "CBAM",
    "CBAM-P2",
    "SAHI_CBAM",
    "SAHI_CBAM-P2",
    "SPD-CBAM",
    "SPD-CBAM-P2",
    "SAHI_SPD-CBAM",
    "SAHI_SPD-CBAM-P2",
]

No_P2 = [
    "Baseline",
    "SPD",
    "CBAM",
    "SPD-CBAM"
]
P2 = [
    "Baseline-P2",
    "SPD-P2",
    "CBAM-P2",
    "SPD-CBAM-P2"
]
SAHI_No_P2 = [
    "SAHI_Baseline",
    "SAHI_SPD",
    "SAHI_CBAM",
    "SAHI_SPD-CBAM"
]
SAHI_P2 = [
    "SAHI_Baseline-P2",
    "SAHI_SPD-P2",
    "SAHI_CBAM-P2",
    "SAHI_SPD-CBAM-P2"
]
best_model = [
    "Baseline",
    "Baseline-P2",
    "SPD-CBAM",
    "SPD-CBAM-P2"
    # "SAHI_SPD-CBAM-P2"
]
step = 3
# folder_path_base = [f"{HOME}\\runs\\detect" ,f"{HOME}\\runs\\predict"]
# output_path = "No_P2_"
# combile_img(No_P2,folder_path_base,output_path,step)
# output_path = "P2_"
# combile_img(P2,folder_path_base,output_path,step)
# output_path = "SAHI_No_P2_"
# combile_img(SAHI_No_P2,folder_path_base,output_path,step)
# output_path = "SAHI_P2"
# combile_img(SAHI_P2,folder_path_base,output_path,step)
output_path = "best_model_No_SAHI"
combile_img(best_model,folder_path_base,output_path,step)

### combile img

In [ ]:
def combile_img(model_name_list,folder_path_base,output_path,num=4):
    all_files = []
    for model in model_name_list:
        if model[0:4] == "SAHI":
            folder_path = os.path.join(folder_path_base[1], model, "visuals")
        else:
            folder_path = os.path.join(folder_path_base[0], model)
        files = sorted(os.listdir(folder_path))
        all_files.append(files)

    file_counts = [len(files) for files in all_files]
    if len(set(file_counts)) != 1:
        print("Folders do not contain the same number of files.")
    else:
        for group_start in range(0, file_counts[0], num):
            group_end = min(group_start + num, file_counts[0])

            for i in range(group_start, group_end):
                group_images = []

                for j, model in enumerate(model_name_list):
                    images = []
                    for k in range(group_start, min(group_end, len(all_files[j]))):
                        file_name = all_files[j][k]
                        if model[0:4] == "SAHI":
                            image_path = os.path.join(
                                folder_path_base[1], model, "visuals", file_name
                            )
                        else:
                            image_path = os.path.join(
                                folder_path_base[0], model, file_name
                            )

                        if os.path.exists(image_path):
                            img = Image.open(image_path)
                            if k == group_start:
                                font = ImageFont.truetype("arial.ttf", 40)

                                text = model
                                d = ImageDraw.Draw(img)
                                text_width, text_height = d.textsize(text, font=font)
                                text_bar_height = text_height + 40
                                text_bar = Image.new(
                                    "RGB", (img.width, text_bar_height), (255, 255, 255)
                                )
                                d = ImageDraw.Draw(text_bar)
                                text_x = (img.width - text_width) / 2
                                d.text((text_x, 10), text, fill=(0, 0, 0), font=font)

                                total_height = img.height + text_bar_height
                                combined = Image.new("RGB", (img.width, total_height))
                                combined.paste(text_bar, (0, 0))
                                combined.paste(img, (0, text_bar_height))
                            else:
                                combined = img
                            images.append(combined)
                        else:
                            print(f"Image not found: {image_path}")
                            break

                    if len(images) == (group_end - group_start):
                        total_height = sum(img.height for img in images)
                        max_width = max(img.width for img in images)
                        new_im = Image.new("RGB", (max_width, total_height))

                        y_offset = 0
                        for im in images:
                            new_im.paste(im, (0, y_offset))
                            y_offset += im.height

                        group_images.append(new_im)

                if len(group_images) == len(model_name_list):
                    total_width = sum(img.width for img in group_images)
                    max_height = max(img.height for img in group_images)
                    final_im = Image.new("RGB", (total_width, max_height))

                    x_offset = 0
                    for im in group_images:
                        final_im.paste(im, (x_offset, 0))
                        x_offset += im.width


                    final_im.save(f"{output_path}_{group_start}.jpg")
